# Codificación avanzada de las variables categóricas. `transformers` personalizados

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set() # Sobreescribe los parámetros de matplotlib
plt.rcParams['figure.figsize'] = [10, 5]

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier


In [ ]:
## Leemos los datos 
fraude = pd.read_csv('../data/01_datos_4_training_cut.txt', sep='|', nrows= 500000)
RS = 20200908
fraude = fraude.sample(frac=1, random_state = 1) 

# Los limpiamos

fraude.drop(['IDTX'], axis = 1, inplace=True)
fraude.FECHATRX = pd.to_datetime(fraude.FECHATRX)

columnas_sin_cambios = ['IDTX', 'FECHATRX','VALOR_TRX']

for columna in fraude.columns:
    if columna not in columnas_sin_cambios:
        fraude[columna] = fraude[columna].astype('category')

### Exploramos
print(fraude.head())
print(fraude.info())

# `transformers`

Recordemos:
- Son objetos de `sklearn` que tienen dos métodos asociados: `fit` y `transform`
- `fit` ajusta los parámetros con los datos que le pasemos
- `transform` transforma los datos que le pasemos usando los parámetros obtenidos con el método `fit`

In [ ]:
from sklearn.preprocessing import StandardScaler

escalado = StandardScaler()
escalado.fit(fraude[['VALOR_TRX']])
transformado = escalado.transform(fraude[['VALOR_TRX']])

print(transformado.mean())
print(transformado.std())

In [ ]:
escalado = StandardScaler()
escalado.fit(fraude[['VALOR_TRX']])


In [ ]:
print(escalado.mean_)
print(escalado.scale_)

# Creando un transformer personalizado


- Los transformers de sklearn son clases basadas en la clase `TransformerMixin`
- Tienen que tener un método __init__, un método `fit` y un método `transform`
- Heredan de `TransformerMixin` el metodo `fit_transform`

> Si queremos usar un transformador personalizado, __tenemos que aprender a crear un objeto de la clase TransformerMixin__ para que pueda ser ensamblado de manera natural con el resto de operaciones de mi algoritmo

In [ ]:
from sklearn.base import TransformerMixin


class MiEscalador(TransformerMixin):
    
    def __init__(self):
        ### Inicializamos los parámetros del transformer
        pass
        
    def fit(self, X, y=None):
        """X es un dataframe de pandas o una numpy array"""
        ## Ajustamos los parámetros del transformer con los datos de entrenamiento
        return self ## Esto siempre tiene que ser así
        
    def transform(self, X, y=None):
        ## Transformamos de acuerdo a los parámetros aprendidos
        return ## Aquí devolverá la numpy array transformada
        

In [ ]:
from sklearn.base import TransformerMixin


class MiEscalador(TransformerMixin):
    
    def __init__(self):
        ### Inicializamos los parámetros del transformer
        self.mean = None
        self.std = None
        pass
        
    def fit(self, X, y=None):
        """X es un dataframe de pandas o una numpy array"""
        ## Ajustamos los parámetros del transformer con los datos de entrenamiento
        self.mean = np.mean(X)
        self.std = np.std(X)
        return self ## Esto siempre tiene que ser así
        
    def transform(self, X, y=None):
        ## Transformamos de acuerdo a los parámetros aprendidos
        return (X-self.mean)/self.std ## Aquí devolverá la numpy array transformada
        

In [ ]:
escalador = MiEscalador()

In [ ]:
x= np.array([1,2,3,4,3,2,4,5])
np.std(x)

In [ ]:
escalador.fit_transform(fraude[['VALOR_TRX']])

In [ ]:
escalador.mean

# Frecuency coding

Esta manera de codificar las variables categóricas _transforma cada categoría en la razón del número de observaciones en esa categoría al número total de observaciones_.

Esto se puede hacer directamente en el dataframe... Pero eso dará problemas antes o después.

La forma adecuada de hacerlo es __creando un transformer personalizado de `sklearn`__ que además podremos ensamblar en nuestras cañerías.



In [ ]:

class FreqEncoder(TransformerMixin):
    
    def __init__(self):
        ### Inicializamos los parámetros del transformer
        self.dicts_frecuencias = {}
        
    def fit(self, X, y=None):
        """X es un dataframe de pandas o una numpy array"""
        ## Ajustamos los parámetros del transformer con los datos de entrenamiento
        
        ## Comprobamos que es una variable del tipo y dimensiones correctas
        X = np.array(X, dtype=str)        
                
        for columna in range(X.shape[1]):
            unique_elements, counts_elements = np.unique(X[:,columna], return_counts=True)
            freqs_elements = counts_elements/counts_elements.sum()
            dict_frecuencias = dict(zip(unique_elements, freqs_elements))
            dict_frecuencias['UNKNOWN_CATEGORY'] = 0
            self.dicts_frecuencias[columna] = dict_frecuencias
        return self
        
    def transform(self, X, y=None):
        ## Transformamos de acuerdo a los parámetros aprendidos
        #assert type(X) == pd.core.frame.DataFrame, 'Necesito un df de pandas'
        #assert X.columns.to_list() == list(self.dicts_frecuencias.keys()), "No coinciden las columnas"
        
        X = np.array(X,dtype=object)

        columnas = []
        for indice in range(X.shape[1]):
            columna = X[:,indice].astype(str)
            transformada = np.where(np.in1d(columna, list(self.dicts_frecuencias[indice].keys())),
                                    columna,
                                    'UNKNOWN_CATEGORY')
            transformada = np.array([self.dicts_frecuencias[indice].get(obs) for obs in transformada.flatten()])
            #print(self.dicts_frecuencias[columna])
            columnas.append(transformada)
                                           
                                           
        return  np.array(columnas).T

In [ ]:
FE = FreqEncoder()
df = fraude[['ENTRYMODE', 'MCC']]
FE.fit(df[0:200])
#FE.dicts_frecuencias

In [ ]:
print(df[800:810])

In [ ]:
FE.transform(df[800:810])

# Top-n encoding

Consiste en reducir el número de categorias, aglomerando las menos comunes en una nueva categoría. Hay varias formas de hacer

- Podemos poner un tope de categorías
- O aglomerar las que no lleven a un umbral
- O aglomerar por percentiles

In [ ]:
fraude.ENTRYMODE.value_counts()

In [ ]:
fraude.ENTRYMODE.value_counts(normalize=True,ascending=False)

In [ ]:
top_n = 3
categorias = fraude.ENTRYMODE.value_counts(normalize=True)
limit_freq = categorias[top_n-1]
mask = categorias<limit_freq
mask

In [ ]:
np.where(fraude.ENTRYMODE.isin(categorias[mask].index),'Other', fraude.ENTRYMODE)[0:20]

In [ ]:
fraude.ENTRYMODE[0:20]

## Ejercicio

- Modificar el `transformer` anterior para crear un `TopNEncoder`

In [ ]:
top_n = 3
categorias = fraude.ENTRYMODE.value_counts(normalize=True)
mask = categorias[:3]
print(mask)
excluded = categorias[3:]
list(excluded.index)

In [ ]:
class TopNEncoder(TransformerMixin):
    
    def __init__(self, top_n=3):
        ### Inicializamos los parámetros del transformer
        self.categories_to_keep = {}
        self.top_n = top_n
        
    def fit(self, X, y=None):
        """X es un dataframe de pandas o una numpy array"""
        ## Ajustamos los parámetros del transformer con los datos de entrenamiento
        
        ## Comprobamos que es una variable del tipo y dimensiones correctas
        X = np.array(X)  
        

        for columna in range(X.shape[1]):
            unique_elements, counts_elements = np.unique(X[:,columna], return_counts=True)
            cuentas = np.array(list(zip(unique_elements, counts_elements)))
            ordenados = cuentas[cuentas[:,-1].argsort()][::-1]
            to_keep = ordenados[0:self.top_n,0]
            self.categories_to_keep[columna] = list(to_keep)
        return self
        


    def transform(self, X, y=None):
        ## Transformamos de acuerdo a los parámetros aprendidos
 
        X = np.array(X)
        columnas = []
        for columna in range(X.shape[1]):
            transformada = np.where(np.in1d(X[:,columna], list(self.categories_to_keep[columna])),
                                    X[:,columna],
                                    'OTHER')
            columnas.append(transformada)
                                           
        return  np.array(columnas).T
    

        


In [ ]:
unique_elements, counts_elements = np.unique(fraude.ENTRYMODE, return_counts=True)
cuentas = np.array(list(zip(unique_elements, counts_elements)))
print(cuentas)
ordenados = cuentas[cuentas[:,-1].argsort()][::-1]
print(ordenados)


In [ ]:
FE = TopNEncoder(5)
df = fraude[['ENTRYMODE', 'MCC']]
FE.fit(df[0:200])
FE.categories_to_keep

In [ ]:
temp = FE.transform(df[0:20])
temp

In [ ]:
print(df[0:20])

# Target encoding

Probablemente la forma más potente de codificar una variable categórica puesto que toma como valores _la media  de la variable objetivo (o cualquier otra función) condicionada a esa categoría_

In [ ]:
(fraude.REPORTE_DE_FRAUDE == 'SI').astype(int)

In [ ]:
class TargetEncoder(TransformerMixin):
    
    def __init__(self):
        ### Inicializamos los parámetros del transformer
        self.dicts_frecuencias = {}
        
    def fit(self, X, y):
        """X es un dataframe de pandas o una numpy array"""
        ## Ajustamos los parámetros del transformer con los datos de entrenamiento
        
        ## Comprobamos que es una variable del tipo y dimensiones correctas
        
        X = np.array(X)
        y = (y == 'SI').astype(int)
        y = np.array(y).reshape(-1,1)
        
        for indice in range(X.shape[1]):
            df = pd.DataFrame(data=X[:,indice], columns = ['X'])
            df['X'] = df['X'].astype(str)
            df[['y']] = y
            df = df.groupby(['X']).mean()
            dict_frecuencias = dict(df['y'])
            dict_frecuencias['UNKNOWN_CATEGORY'] = np.array(y).mean()
            self.dicts_frecuencias[indice] = dict_frecuencias
        return self
        
    def transform(self, X, y=None):
        ## Transformamos de acuerdo a los parámetros aprendidos
        
        X = np.array(X)
        columnas = []
        for indice in range(X.shape[1]):
            columna = X[:,indice].astype(str)
            transformada = np.where(np.in1d(columna, list(self.dicts_frecuencias[indice].keys())),
                                    columna,
                                    'UNKNOWN_CATEGORY')
            #print(transformada)
            transformada = np.array([self.dicts_frecuencias[indice].get(obs) for obs in transformada.flatten()])
            #print(self.dicts_frecuencias[columna])
            columnas.append(transformada)
                                           
        return  np.array(columnas).T        



In [ ]:
np.array([1,2,3]).astype(str)

In [ ]:
TE = TargetEncoder()
df = fraude[['ENTRYMODE', 'MCC']]
TE.fit(df, fraude.REPORTE_DE_FRAUDE)
TE.dicts_frecuencias

In [ ]:
TE.transform(df)

In [ ]:
df

## Suavizando y generalizando el TargetEncoding 

Es muy sencillo modificar nuestro transformer para que no obtenga la media de la variable objetivo sino cualquier función que le queramos pasar (mediana, máximo, mínimo, etc...)
Además, podemos añadir un parámetro de suavizamiento para controla el OF.
Con este transformer tenemos un método muy potente para transformar las columnas categóricas.

In [ ]:
class TargetEncoder(TransformerMixin):
    
    def __init__(self, agg_func = np.mean, mix_param = 0.98):
        ### Inicializamos los parámetros del transformer
        self.dicts_frecuencias = {}
        self.agg_func = agg_func
        self.mix_param = mix_param
        
    def fit(self, X, y):
        """X es un dataframe de pandas o una numpy array"""
        ## Ajustamos los parámetros del transformer con los datos de entrenamiento
        
        ## Comprobamos que es una variable del tipo y dimensiones correctas
        
        X = np.array(X)
        y = (y == 'SI').astype(int)
        y = np.array(y).reshape(-1,1)
    
        for indice in range(X.shape[1]):
            df = pd.DataFrame(data=X[:,indice], columns = ['X'])
            df['X'] = df['X'].astype(str)
            df[['y']] = y
            df = df.groupby(['X']).agg(self.agg_func)
            df[['y']] = self.mix_param*df[['y']] + (1-self.mix_param)*np.array(y).mean()
            #print(np.array(y).mean())
            #print(df)
            
            dict_frecuencias = dict(df['y'])
            dict_frecuencias['UNKNOWN_CATEGORY'] = np.array(y).mean()
            self.dicts_frecuencias[indice] = dict_frecuencias
        return self

    
        
    def transform(self, X, y=None):
        ## Transformamos de acuerdo a los parámetros aprendidos
        
        X = np.array(X)
        columnas = []
        for indice in range(X.shape[1]):
            columna = X[:,indice].astype(str)
            transformada = np.where(np.in1d(columna, list(self.dicts_frecuencias[indice].keys())),
                                    columna,
                                    'UNKNOWN_CATEGORY')
            transformada = np.array([self.dicts_frecuencias[indice].get(obs) for obs in transformada.flatten()])
            columnas.append(transformada)
                                           
        return  np.array(columnas).T        

                                           
        return  np.array(columnas).T 

In [ ]:
TE = TargetEncoder(agg_func = np.mean, mix_param = 0.95)
df = fraude[['ENTRYMODE', 'MCC']]
TE.fit(df, fraude.REPORTE_DE_FRAUDE)
TE.dicts_frecuencias

In [ ]:
TE.transform(df)

## Incorporando los codificadores al algoritmo

In [ ]:
#fraude.REPORTE_DE_FRAUDE = (fraude.REPORTE_DE_FRAUDE == 'SI').astype(int)

# Definimos los predictores
predictores_num = [fraude.columns[-2]]
predictores_dummy = list(fraude.columns[i] for i in [3,4,5,6,13,14,15,17])
predictores_target = list(fraude.columns[i] for i in [0,1,2,7,9,10,12,16])


# Definimos los vectores de predictores y la respuesta
y = fraude['REPORTE_DE_FRAUDE']
X = fraude[predictores_num + predictores_dummy + predictores_target]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('BoxCox',  PowerTransformer(method='yeo-johnson')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

target_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('target_coding', TargetEncoder(mix_param = 0.95))])

preprocesado = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, predictores_num),
        ('cat', categorical_transformer, predictores_dummy),
        ('imp', target_transformer, predictores_target)])

# Definimos la tubería

steps = [('feat_prepro', preprocesado), 
         ('predictor', RandomForestClassifier(n_jobs=-1))]

pipe = Pipeline(steps)


pipe.fit(X_train, y_train)


In [ ]:
def score(mod, X_test, y_test, positive_class = 'SI', alarm = None):
    if not alarm:
        alarm = sum(y_test == positive_class)/len(y_test)
    y_pred_prob = mod.predict_proba(X_test)[:,1]
    resultados = pd.DataFrame({'Prob':y_pred_prob, 'Label':y_test.values})
    resultados.sort_values('Prob', axis=0, ascending=False, inplace=True)
    resultados.reset_index(inplace=True)
    alarmas = int(alarm*len(y_test))
    print('Casos Analizados:{}'.format(len(y_test)))
    print('Alarmas:{}'.format(alarmas))
    print('Cazados:{}'.format(sum(resultados[0:alarmas].Label==positive_class)))
    print('Fraude total en el conjunto:{}'.format(sum(resultados.Label == positive_class)))
    print('Recall:{}'.format(sum(resultados[0:alarmas].Label==positive_class)/sum(resultados.Label == positive_class)))
    return


score(pipe, X_test, y_test)